<a href="https://colab.research.google.com/github/ellozam/ciencia-datos-notebooks/blob/main/proyecto3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ✅ Paso 1: Instalar Gradio si no lo tenés
!pip install gradio --quiet

# ✅ Paso 2: Leer CSV directamente desde Colab (ya descomprimido)
import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
import glob

# ✅ Buscar el archivo CSV automáticamente
csv_path = glob.glob("/content/API_*.csv")
assert csv_path, "❌ No se encontró ningún archivo CSV que empiece con 'API_' en /content"
csv_file = csv_path[0]

# ✅ Leer y preparar los datos
df = pd.read_csv(csv_file, skiprows=4)

# ✅ Filtrar columnas necesarias
columnas = ['Country Name'] + [str(year) for year in range(2000, 2023)]
df = df[columnas].dropna()
paises = sorted(df['Country Name'].unique().tolist())

# ✅ Función para mostrar datos
def visualizar_pib(pais):
    datos = df[df['Country Name'] == pais].drop(columns='Country Name').T
    datos.columns = ['PIB']
    datos.index.name = 'Año'
    datos = datos.dropna()
    datos.index = datos.index.astype(int)
    datos['Crecimiento %'] = datos['PIB'].pct_change() * 100
    tabla = datos.round(2).reset_index()

    # Gráfico
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(datos.index, datos['PIB'] / 1e9, marker='o', color='blue')
    ax.set_title(f'PIB de {pais} (2000–2022)')
    ax.set_xlabel('Año')
    ax.set_ylabel('PIB (miles de millones USD)')
    ax.grid(True)
    plt.tight_layout()

    return fig, tabla

# ✅ Interfaz Gradio
with gr.Blocks() as demo:
    gr.Markdown("## 🌍 Visualizador de PIB Mundial (2000–2022)")
    dropdown = gr.Dropdown(choices=paises, label="Seleccioná un país", value=paises[0])
    boton = gr.Button("Mostrar análisis")
    grafico = gr.Plot()
    tabla = gr.Dataframe()

    boton.click(fn=visualizar_pib, inputs=dropdown, outputs=[grafico, tabla])

# ✅ Lanzar Gradio
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c473ae89be43b59a51.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
